## Swarm Team  

The `Swarm` team is inspired by the one provided by [OpenAI](https://github.com/openai/swarm), where an agent can choose to hand off a task to another agent.  

It functions as a group chat, similar to the ones we've seen before, where each agent broadcasts their responses to all other agents in the group. 
However, the next agent to act is determined by the most recent `HandsoffMessage`.  

**NOTE**: The handoff functionality relies on the tool calling feature, meaning the LLM used by the agent **MUST** support tool calling.  

In [6]:
import sys
sys.path.append("..")

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import Swarm
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.ui import Console
from model_clients.azure import get_model
from rich import print


model_client = get_model()

As usual, let's now define the tools involved

In [7]:
def refund_flight(flight_id: str) -> str:
    """Refund a flight

    :param flight_id: The ID of the flight.
    :return: The confirmation that the flight has been refunded.
    """
    print(f"[blue]Refunding flight {flight_id}[/blue]")
    return f"Flight {flight_id} refunded"


Now its time to define the agents...

In [8]:
travel_agent = AssistantAgent(
    "travel_agent",
    model_client=model_client,
    handoffs=["flights_refunder", "user"],
    system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you can handoff to the user.
    Use TERMINATE when the travel planning is complete.""",
)

flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=model_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight reference numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.
    Use TERMINATE when the refunding is complete or when the bookins id is 'exit'.""",
)

user_agent= UserProxyAgent(name="user",
                            description= "The user in charge of handling additional requests from the assistant",input_func=input)

And finally the Swarm team with the proper termination conditions

In [10]:
termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE")
team = Swarm([travel_agent, flights_refunder], termination_condition=termination)


Now run the workflow, you can see the handsoff in action using, as example:  

- 'I need help with my flight'
- 'I need a refund'
- 'I want my flight 678 refunded'

And test the different behavior

In [11]:

# This is the task that the user wants to accomplish.
task = input("How can the agent help you?: ")

# Run the task
task_result = await Console(team.run_stream(task=task))

print(task_result.messages)
# Get the last message because it might be a handoff message to the user
last_message = task_result.messages[-1]

while isinstance(last_message, HandoffMessage) and last_message.target == "user":
    user_message = input("User: ")
    # Note that the user message is sent to the last agent that handed off to the user (via source)
    # And since the Team keeps its state, the previous information is still available.
    task_result = await Console(
        team.run_stream(task=HandoffMessage(source="user", target=last_message.source, content=user_message))
    )
    last_message = task_result.messages[-1]

---------- user ----------
TERMINATE


[TextMessage(source='user', models_usage=None, metadata={}, content='TERMINATE', type='TextMessage')]